In [ ]:
# Install
!pip install imutils -q

In [ ]:
#Import packages
import numpy as np 
import pandas as pd
import os
import cv2
from skimage import measure
from glob import glob
import imutils
import matplotlib.pyplot as plt

from imutils import contours
from PIL import Image as Img
from IPython.display import Image

from joblib import Parallel, delayed
from tqdm import tqdm_notebook as tqdm
from tqdm.notebook import tqdm
from skimage import measure

In [ ]:
# Mount to Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
class CFG:

    raw_path = '/content/drive/MyDrive/DL-T Cell Segmentatiom/data_for_classification/'
    mask_path = '/content/drive/MyDrive/DL-T Cell Segmentatiom/imagej_for_classification/'
    imagej_out_path = '/content/drive/MyDrive/DL-T Cell Segmentatiom/imagej_patch_for_classification/'
    
# if not os.path.exists(CFG.train_out_path):
#     os.makedirs(CFG.train_out_path)
#if not os.path.exists(CFG.test_out_path):
    #os.makedirs(CFG.test_out_path)

### Raw image multiply mask

In [ ]:
# counts the number of raw and mask images, make sure they have same number of images
filenames = glob(os.path.join(CFG.raw_path + '/*.jpg'))
print(len(filenames))

jnames = glob(os.path.join(CFG.mask_path + '/*.jpg'))
print(len(jnames))

0
0


In [ ]:
filenames_pre = []
jnames_pre = []
for i in range(len(filenames)):
  filenames_pre.append(filenames[i].split('/')[-1][:-4])

for i in range(len(jnames)):
  jnames_pre.append(jnames[i].split('/')[-1][:-9])

In [ ]:
def return_w_hw(h, w, th, tw):
  """
  this function calculates the pixel dimensions need to be paded in height and width
  h: height after padding (output size)
  w: width after padding (outputs size)
  th: height before padding (input size)
  tw: width before padding (input size)
  """
  
    if h - th < 0:
        top = bottom = 0
    elif (h-th)%2 == 0:
        top = (h-th)/2
        bottom = top
    else:
        top = (h-th)//2
        bottom = top + 1


    if w - tw < 0:
        left = right = 0   
    elif (w - tw)%2 == 0:
        left = (w - tw)/2
        right = left
    else:
        left = (w - tw)//2
        right = left + 1

    return int(top), int(bottom), int(left), int(right)

In [ ]:
# modify this directory according to your data path
maskj_dir = '/content/drive/MyDrive/DL-T Cell Segmentatiom/imagej_for_classification/'

def create_patches(image_name, outdir, outdirj):
  """
  this function creats patches and stores patch images
  the processing is implemented as follows:
  1) generate the multiplied images
  2) generate the contour for each patch
  3) remove contours that are too small (noise) or too large (not single cell)
  4) find dimensions needed to be zero-padded for each patch
  5) zero padding each selected region to 100*100
  6) resize all the patches to 224*224, according to the input size of the pretrained classification model
  7) save the patches with name: "[IMAGE NAME]_p[ID].jpg"
  """
    counts = 0

     
    mask_name = maskj_dir + image_name.split('/')[-1][:-4] + '_mask.jpg'
    # print(mask_name)
    image = cv2.imread(image_name, 0)
    # print(image)
    maskj = cv2.imread(mask_name, 0)
    maskj = np.array(maskj)/255
    # print(maskj)
    multiplied = np.multiply(image, maskj)
    # print(multiplied)
    labelsj = measure.label(maskj, background = 0)
  
    # remember to comment all command to plot figures when parallel processing
    # open too many figure windows will cause crash
    plt.figure()
    plt.imshow(image)
    plt.figure()
    plt.imshow(multiplied)

    mask_mul = np.zeros(image.shape, dtype="uint8")
    for label in np.unique(labelsj):
        # print(label)
        labelMask = np.zeros(image.shape, dtype="uint8")
        labelMask[labelsj == label] = 255
        numPixels = cv2.countNonZero(labelMask)
        # filter out noise and regions that are too large (most probably not single cell if > 3000)
        if numPixels > 300 and numPixels < 3000:
            mask_mul = cv2.add(mask_mul, labelMask)
            counts = counts + 1
    # print(counts)
    # plt.figure()
    # plt.imshow(mask)

    bbox_list = []
    box_width = []
    box_height = []

    cnts_j = cv2.findContours(mask_mul.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts_j = imutils.grab_contours(cnts_j)
    cnts_j = contours.sort_contours(cnts_j)[0]

    # zero padding to 100*100
    bb_dim = 100

    for (i, c) in enumerate(cnts_j):
        (x, y, w, h) = cv2.boundingRect(c)
        pad_dim = max(w, h)
        bbox_imgj = multiplied[y:y+pad_dim, x:x+pad_dim]
        # pad to 100*100
        pad_top, pad_bottom, pad_left, pad_right = return_w_hw(bb_dim, bb_dim, pad_dim, pad_dim)
        bbox_imgj = np.pad(bbox_imgj, [(pad_top, pad_bottom), (pad_left, pad_right)], constant_values=0)
        # resize to 224*224
        bbox_imgj = cv2.resize(bbox_imgj, (224,224))
        
        plt.figure()
        plt.imshow(bbox_img_pad)
        # print(bbox_img_pad.shape)
        # display(Img.fromarray((bbox_imgj).astype(np.uint8)))
        cv2.imwrite(outdirj + image_name.split('/')[-1][:-4] + '_p'+ str(i) + '.jpg', bbox_imgj)
  
    

In [ ]:
# you can test the function using a single image
# plot the figures you want to see the difference before and after multiply the imageJ mask; and the patches generated
create_patches(filenames[10], CFG.raw_out_path, CFG.imagej_out_path)

IndexError: list index out of range

In [ ]:
# parallel processing
# modify the [filenames] and [total] if needed
res2 = Parallel(n_jobs=16, backend='threading')(delayed(
    create_patches)(i, CFG.raw_out_path, CFG.imagej_out_path) for i in tqdm(filenames, total=len(filenames)))

In [ ]:
# check the number of output patches to see if processing was successful
j_patch = glob(os.path.join(CFG.imagej_out_path + '/*.jpg'))
print(len(j_patch))

10055
8855
